In [1]:
import torch
import torchsummary

In [2]:
import numpy as np

In [3]:
from torch import nn

In [4]:
layer1 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1)

In [5]:
batch_norm1 = nn.BatchNorm2d(num_features=64)

In [6]:
prelu1 = nn.PReLU()

In [7]:
layer2 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=3,padding=1)

In [8]:
batch_norm2 = nn.BatchNorm2d(num_features=64)

In [11]:
class ResBlock(nn.Module):
    def __init__(self,kernel_size,channels,padding,stride):
        super(ResBlock,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=channels,out_channels=channels,kernel_size=kernel_size,padding=padding,stride=stride)
        self.batch_norm1 = nn.BatchNorm2d(num_features=channels)
        self.prelu1 = nn.PReLU()
        self.conv2 = nn.Conv2d(in_channels=channels,out_channels=channels,kernel_size=kernel_size,padding=padding,stride=stride)
        self.batch_norm2 = nn.BatchNorm2d(num_features=channels)
    def forward(self,inp):
        x = self.conv1(inp)
        x = self.batch_norm1(x)
        x = self.prelu1(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        output = x + inp
        return output

In [15]:
resblock = ResBlock(3,64,1,1)

In [13]:
inp = torch.randn(1,64,2,2)

In [17]:
resblock(inp).shape

torch.Size([1, 64, 2, 2])

In [26]:
class SubPixelConv(nn.Module):
    def __init__(self,kernel_size,in_channels,out_channels,padding,stride):
        super(SubPixelConv,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=in_channels,out_channels=out_channels,kernel_size=kernel_size,padding=padding,stride=stride)
        self.pixel_shuffler1 = nn.PixelShuffle(2)
        self.prelu1 = nn.PReLU()
    def forward(self,inp):
        x = self.conv1(inp)
        x = self.pixel_shuffler1(x)
        output = self.prelu1(x)
        return output

In [29]:
subpixelconv(resblock(inp)).shape

torch.Size([1, 64, 4, 4])

In [73]:
class Generator(nn.Module):
    def __init__(self):
        super(SubPixelConv,self).__init__()
        self.model = nn.Sequential()
        self.model.add_module(module=nn.Conv2d(in_channels=3,out_channels=64,kernel_size=9,padding=4,stride=1),name='conv1')
        self.model.add_module(module=nn.PReLU(),name='prelu1')
        for i in range(16):
            self.model.add_module(module=ResBlock(3,64,1,1),name='res'+str(i+1))

In [74]:
generator = Generator()

In [75]:
generator.model

Sequential(
  (conv1): Conv2d(3, 64, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4))
  (prelu1): PReLU(num_parameters=1)
  (res1): ResBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu1): PReLU(num_parameters=1)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res2): ResBlock(
    (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (prelu1): PReLU(num_parameters=1)
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (batch_norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (res3): ResBlock(
    (conv1): Conv2d(

In [76]:
torchsummary.summary(generator.model,input_size=(3,96,96))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 96, 96]          15,616
             PReLU-2           [-1, 64, 96, 96]               1
            Conv2d-3           [-1, 64, 96, 96]          36,928
       BatchNorm2d-4           [-1, 64, 96, 96]             128
             PReLU-5           [-1, 64, 96, 96]               1
            Conv2d-6           [-1, 64, 96, 96]          36,928
       BatchNorm2d-7           [-1, 64, 96, 96]             128
          ResBlock-8           [-1, 64, 96, 96]               0
            Conv2d-9           [-1, 64, 96, 96]          36,928
      BatchNorm2d-10           [-1, 64, 96, 96]             128
            PReLU-11           [-1, 64, 96, 96]               1
           Conv2d-12           [-1, 64, 96, 96]          36,928
      BatchNorm2d-13           [-1, 64, 96, 96]             128
         ResBlock-14           [-1, 64,

In [35]:
model.forward(inp)

tensor([[[[ 1.3964, -0.2138],
          [ 0.7101, -1.4748]],

         [[-0.6391,  1.4618],
          [ 1.2519,  0.0663]],

         [[-0.3244, -0.7537],
          [-1.5978, -0.4525]],

         [[ 0.4607,  0.1563],
          [ 1.0554, -2.6950]],

         [[-0.9548, -0.1770],
          [ 0.2713, -0.7973]],

         [[ 2.3619,  0.3170],
          [-0.9167, -0.2072]],

         [[ 0.0494, -2.0885],
          [ 0.3592, -0.2938]],

         [[ 0.5089,  0.4162],
          [ 0.2064, -1.8366]],

         [[-0.4067, -0.0246],
          [-1.3946, -0.6146]],

         [[ 0.3186, -1.3618],
          [ 2.6360,  0.5912]],

         [[-2.0839, -0.0138],
          [ 2.6350,  0.1775]],

         [[ 1.3910, -0.7048],
          [ 0.1877, -1.1083]],

         [[ 0.6507,  0.3739],
          [ 0.0545,  0.2233]],

         [[-0.1060,  2.7515],
          [ 0.7918, -0.2241]],

         [[-0.3089, -1.1391],
          [-0.0386,  0.6891]],

         [[-1.0344, -1.2680],
          [-0.0309, -1.0123]],

        

In [44]:
resblock = list(model.children())[0]

In [47]:
resblock.forward(inp) - model.forward(inp)

tensor([[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 

In [51]:
layers = list(resblock.children())

In [52]:
x = inp
for layer in layers:
    x = layer(x)

In [54]:
x+inp - resblock.forward(inp)

tensor([[[[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 0.,  0.]],

         [[ 0.,  0.],
          [ 